# DARPA Resting State fMRI Analysis Tutorial

## Step 1: Preprocessing fMRI Data
This section covers ______.

## Step 3: Generate Nuisance Regressors for 
This section creates custom nuisance regressors for use in the first-level analysis. Specifically, the script creates (1) demeaned, detrended, and orthogonalized motion regressors that explain 90% of the variance in the motion; and (2) the timepoints to censor based on functional displacement (FD; Power et al. 2012, 2014) values. Users will specify which FD thresholds to use to create subject-specific timepoint censoring files to be used in the first level analyses. 

In [1]:
import os
import numpy as np
import pylab as plt
from pandas import read_csv
from scipy.io import savemat
from scipy.stats import probplot
from scipy.signal import detrend
from sklearn.decomposition import PCA

# subjlist = ['hc001']

rs_params = '/autofs/space/lilli_003/users/DARPA-RestingState/RS_parameters/rs_subjects_12-2-18.txt'

with open (rs_params) as subjlist:
    
    for subject in subjlist:
        
        subject = subject.strip()

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Define parameters.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

        ## Define acquisition parameters.    
        n_acq = 124
        tr = 3

        ## Scrubbing parameters.
        thresholds = [0.0, 0.5, 1.0, 1.5]

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Compute framewise displacement.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

        ## Read motion data.
        out_dir = '/autofs/space/lilli_003/users/DARPA-RestingState/%s/rest_001/001' %subject
        mc = os.path.join(out_dir, 'fmcpr.mcdat')
        mc = np.loadtxt(mc)[:,1:7]

        ## Invert angular displacement.
        fd = mc.copy()
        fd[:,:3] = np.deg2rad(fd[:,:3]) 
        fd[:,:3] *= 50

        ## Compute framewise displacement (See Power 2012, 2014).
        fd = np.insert( np.abs( np.diff(fd, axis=0) ).sum(axis=1), 0, 0 )

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Compute motion regressors.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

        ## Remove trends.
        mc = detrend(mc, axis=0, type='constant')
        mc = detrend(mc, axis=0, type='linear')

        ## Perform PCA.
        pca = PCA(n_components=6)
        mc = pca.fit_transform(mc)

        ## Take only the number of components explaining 90% of the variance.
        varexp = np.cumsum(pca.explained_variance_ratio_)
        n_components = np.argmax(varexp >= 0.9) + 1
        mc = mc[:,:n_components]

        ## Save motion regressor.
        f = os.path.join(out_dir, 'rest.mc.txt')
        np.savetxt(f, mc, fmt='%s')

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Write scrubbers.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

        ## Define TR onsets.
        tr_onsets = np.insert( np.cumsum( np.ones(n_acq - 1) * tr ), 0, 0 )

        for threshold in thresholds:

            ## Find threshold violations.
            if not threshold: ix, = np.where(fd >= np.inf)
            else: ix, = np.where(fd >= threshold)

            ## Save.
    #         f = os.path.join(out_dir, 'rest.censor.%s.par' %threshold)
            f = os.path.join(out_dir, 'rest.censor.par')
            if len(ix): np.savetxt(f, tr_onsets[ix,np.newaxis], fmt='%s')

        print 'Done.'

Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


## Step 4: Decide on Censor Level
This section will create a summary file for specified subjects and FD thresholds describing how many volumes (acquisitions) will be censored if the user chooses a specific FD threshold. The summary file is written to the fMRI folder.

In [ ]:
import os
from pandas import DataFrame, Series

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Define subjects.
subjects = ['hc001']

## Scrubbing parameters.
thresholds = [0.0, 0.5, 1.0, 1.5]

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Make summary file.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

columns = ['Subject'] + ['FD=%s' %fd for fd in thresholds]
df = DataFrame([],columns=columns)

for subject in subjects:
    
    ## Initialize.
    info = Series()
    info['Subject'] = subject
    
    ## Iteratively lookup and store information.
    for fd in thresholds:
        f = '/autofs/space/lilli_003/users/DARPA-RestingState/%s/RS.censor.par' %(subject)
        if os.path.isfile(f): info['FD=%s' %fd] = len(open(f,'r').readlines())
        else: info['FD=%s' %fd] = 0
            
    ## Append information.
    df = df.append(info, ignore_index=True)
    
## Save.
df.to_csv('fmri/msit_censor_summary.csv', index=False)
print 'Done.'

## Step 3: Visualize using MMVT